In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.

In [ ]:
!pip install -U datasets

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes  #newline

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate #newline

In [ ]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig, QuantoConfig
from trl import SFTTrainer
from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
from trl import setup_chat_format

In [ ]:
# Loading the model with double quantization
#model_name = "PY007/TinyLlama-1.1B-step-50K-105b"
model_name = "TinyLlama/TinyLlama-1.1B-step-50K-105b"
#model_name = "EleutherAI/gpt-neo-125m"
#model_name = "openai-community/gpt2"
#model_name = "openai-community/gpt2-medium"
#model_name = "openai-community/gpt2-large"
#model_name = 'facebook/opt-350m'
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",                  #new line
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token  # modification

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


In [ ]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='openai-community/gpt2-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
#model, tokenizer = setup_chat_format(model, tokenizer, resize_to_multiple_of = 64)


In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.bos_token = tokenizer.eos_token

In [ ]:
tokenizer.bos_token_id = tokenizer.eos_token_id

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='facebook/opt-350m', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
tokenizer.pad_token_id

50256

In [ ]:
tokenizer.all_special_tokens

['<|endoftext|>']

In [ ]:
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
messages = [
    {
        "role": "user",
        "content": "You are a friendly chatbot who always responds is a polite manner",
    },
    {"role": "assistant", "content": "Hey, do you think it is okay to not exercise daily for humans?"}
]

In [ ]:
text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
text

'<|user|>\nYou are a friendly chatbot who always responds is a polite manner<|endoftext|>\n<|assistant|>\nHey, do you think it is okay to not exercise daily for humans?<|endoftext|>\n'

In [ ]:
text = "You are a friendly chatbot who always responds is a polite manner<|endoftext|>Hey, do you think it is okay to not exercise daily for humans?<|endoftext|>"

In [ ]:
decoded_text = tokenizer(text)
decoded_text = tokenizer.decode(decoded_text['input_ids'], skip_special_tokens = False)
decoded_text

'<|user|>\nYou are a friendly chatbot who always responds is a polite manner<|endoftext|>\n<|assistant|>\nHey, do you think it is okay to not exercise daily for humans?<|endoftext|>\n'

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
from datasets import load_dataset

dataset = load_dataset("daily_dialog")

Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.remove_columns(["act", "emotion"])
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog'],
        num_rows: 1000
    })
})

In [ ]:
def preprocess_function(Data, tokenize, add_generation_prompt):
  processed_data = []

  for dialogs in Data:
    flag = 0
    #wrapped_dialogues = [{"role" : "system", "content" : ''}]
    wrapped_dialogues = []

    for dialog in dialogs['dialog']:
      if flag == 0:
        wrapped_dialog = {"role" : "user", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 1
      else:
        wrapped_dialog = {"role" : "assistant", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 0

    wrapped_dialogues = tokenizer.apply_chat_template(wrapped_dialogues, tokenize=False)
    processed_data.append(wrapped_dialogues)
    output  = pd.DataFrame({"inputs": processed_data})

  return output

In [ ]:
import pandas as pd

In [ ]:
train_ds = preprocess_function(dataset['train'], tokenize=False, add_generation_prompt=False)
train_ds

,inputs
0,"<|user|>\nSay , Jim , how about going for a fe..."
1,<|user|>\nCan you do push-ups ? <|endoftext|>\...
2,<|user|>\nCan you study with the radio on ? <|...
3,<|user|>\nAre you all right ? <|endoftext|>\n<...
4,"<|user|>\nHey John , nice skates . Are they ne..."
...,...
11113,"<|user|>\nHello , I bought a pen in your shop ..."
11114,<|user|>\nDo you have any seats available ? <|...
11115,"<|user|>\nUncle Ben , how did the Forbidden Ci..."
11116,"<|user|>\nMay I help you , sir ? <|endoftext|>..."


In [ ]:
tokenizer.model_max_length

1024

In [ ]:
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 1024

In [ ]:
val_ds = preprocess_function(dataset['validation'], tokenize=False, add_generation_prompt=False)
val_ds

,inputs
0,"<|user|>\nGood morning , sir . Is there a bank..."
1,<|user|>\nGood afternoon . This is Michelle Li...
2,<|user|>\nWhat qualifications should a reporte...
3,"<|user|>\nHi , good morning , Miss ? what can ..."
4,"<|user|>\nExcuse me , ma'am . Can you tell me ..."
...,...
995,"<|user|>\nHello , who is speaking ? <|endoftex..."
996,<|user|>\nAhh ... What a fine day ! I do feel ...
997,"<|user|>\nI'm so sorry about your brother , Mr..."
998,"<|user|>\nHi , Jeny.Are still working ? <|endo..."


In [ ]:
frames = [train_ds, val_ds]
train_ds = pd.concat(frames)
train_ds

,inputs
0,"<|user|>\nSay , Jim , how about going for a fe..."
1,<|user|>\nCan you do push-ups ? <|endoftext|>\...
2,<|user|>\nCan you study with the radio on ? <|...
3,<|user|>\nAre you all right ? <|endoftext|>\n<...
4,"<|user|>\nHey John , nice skates . Are they ne..."
...,...
995,"<|user|>\nHello , who is speaking ? <|endoftex..."
996,<|user|>\nAhh ... What a fine day ! I do feel ...
997,"<|user|>\nI'm so sorry about your brother , Mr..."
998,"<|user|>\nHi , Jeny.Are still working ? <|endo..."


In [ ]:
test_ds = preprocess_function(dataset['test'], tokenize=False, add_generation_prompt=False)
test_ds

,inputs
0,"<|user|>\nHey man , you wanna buy some weed ? ..."
1,<|user|>\nThe taxi drivers are on strike again...
2,<|user|>\nWe've managed to reduce our energy c...
3,"<|user|>\nBelieve it or not , tea is the most ..."
4,<|user|>\nWhat are your personal weaknesses ? ...
...,...
995,"<|user|>\nFrank ’ s getting married , do you b..."
996,"<|user|>\nOK . Come back into the classroom , ..."
997,<|user|>\nDo you have any hobbies ? <|endoftex...
998,"<|user|>\nJenny , what's wrong with you ? Why ..."


In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict


tdf = train_ds
#vdf = df_val
testdf = test_ds
tds = Dataset.from_pandas(tdf)
#vds = Dataset.from_pandas(vdf)
testdf = Dataset.from_pandas(testdf)


ds = DatasetDict()

ds['train'] = tds
#ds['validation'] = vds
ds['test'] = testdf
print(ds)

DatasetDict({
    train: Dataset({
        features: ['inputs', '__index_level_0__'],
        num_rows: 12118
    })
    test: Dataset({
        features: ['inputs'],
        num_rows: 1000
    })
})


In [ ]:
train_dataset = ds['train']
eval_dataset = ds['test']

In [ ]:
model, tokenizer = setup_chat_format(model, tokenizer, resize_to_multiple_of = 64)

In [ ]:
tokenizer.all_special_tokens

['<|im_start|>', '<|im_end|>', '<|endoftext|>']

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<|user|>', "<|assistant|>"]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

We have added 2 tokens


In [ ]:
tokenizer.all_special_tokens

['<|im_start|>', '<|im_end|>', '<|endoftext|>', '<|user|>', '<|assistant|>']

In [ ]:
# Setting arguments for low-rank adaptation

#model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# Setting training arguments
output_dir = "SSahas/openai_community_med_e6"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
#optim = "paged_adamw_32bit"
optim = "adamw_hf"
num_train_epochs = 4
save_strategy="epoch"
evaluation_strategy = 'steps'
#save_steps="epoch"
#save_steps = 350
logging_steps = 200
learning_rate = 9e-4
#max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 700     # Number of training steps
#warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_strategy = save_strategy,
    #evaluation_strategy = evaluation_strategy,
    #save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    #max_grad_norm=max_grad_norm,
    #max_steps=max_steps,
    #warmup_ratio=warmup_ratio,
    #lr_scheduler_type=lr_scheduler_type,
    push_to_hub=False,
    report_to='none'
)

In [ ]:
max_seq_length = tokenizer.model_max_length
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset = eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    dataset_text_field='inputs',
    tokenizer=tokenizer,
    args=training_arguments,
    #neftune_noise_alpha=5
)
peft_model.config.use_cache = False

Map:   0%|          | 0/12118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,6.610600
400,2.157500
600,2.168500
800,2.111200
1000,2.118600
1200,2.061500
1400,2.095200
1600,2.072700
1800,2.063100
2000,2.062500


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=48472, training_loss=1.8960516457904741, metrics={'train_runtime': 4271.6558, 'train_samples_per_second': 11.347, 'train_steps_per_second': 11.347, 'total_flos': 1.3602720526467072e+16, 'train_loss': 1.8960516457904741, 'epoch': 4.0})

In [ ]:
messages = [
    #{"role" : "system", "content" : ''},
    {"role": "user", "content": "Hello, My name is Sahas."},
    {"role": "assistant", "content": "Hi, Sahas. How are you?"},
    {"role": "user", "content": "I am Great, How are you?"},
    {"role": "assistant", "content": "I am fine."},
    {"role": "user", "content": "what is my nam?"},
]

In [ ]:
#prompt = tokenizer.apply_chat_template(messages, tokenize = False)
#input_ids = tokenizer(prompt, return_tensors = "pt").input_ids
#input_ids = tokenizer(prompt , return_tensors = "pt").to("cuda")
input_ids = tokenizer.apply_chat_template(messages, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")

peft_model_result = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=15,eos_token_id = 50256))
peft_model_text_output = tokenizer.decode(peft_model_result[0], skip_special_tokens=True)
print(peft_model_text_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|user|>
Hello, My name is Sahas.
<|assistant|>
Hi, Sahas. How are you?
<|user|>
I am Great, How are you?
<|assistant|>
I am fine.
<|user|>
what is my nam?
<|assistant|>
 My name is Wang Lin.How are you?     


In [ ]:
input_ids = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=True)
input_ids

In [ ]:
!zip -r /kaggle/working/openai_gpt_large_model.zip /kaggle/working/openai_gpt_med_r64

In [ ]:
trainer.save_model("./openai_gpt_med_r64")

In [ ]:
!pip install huggingface_hub


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
trainer.push_to_hub("SSahas/openai_community_med_r32_a32_E3, loss: 1.59 ")

training_args.bin:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SSahas/openai_community_med_e3/commit/c20639554eaf2d1fa83d1bb3f961c547b6eeadb8', commit_message='SSahas/openai_community_med_r32_a32_E3, loss: 1.59 ', commit_description='', oid='c20639554eaf2d1fa83d1bb3f961c547b6eeadb8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!pip install evaluate rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a6d11738ab83f88b93318800a75f295b3fe06f1460ea84bec048a90d412ed09
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
#tokenizer.model_max_length

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

In [ ]:
def preprocess_function(Data, tokenize, add_generation_prompt):
  processed_data = []
  label_data  = []
  original_labels = []
  for dialogs in Data:
    flag = 0
    wrapped_dialogues = []
    labels = []
    for dialog in dialogs['dialog']:
      if flag == 0:
        wrapped_dialog = {"role" : "user", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 1
      else:
        wrapped_dialog = {"role" : "assistant", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 0
    if wrapped_dialogues[-1]['role'] == "assistant":
      #label = [{"role": "assisitant", "content" : wrapped_dialogues[-1]['content']}]
      label = str(wrapped_dialogues[-1]['content'])
      original_labels.append(wrapped_dialogues[-1]['content'])
      wrapped_dialogues = wrapped_dialogues[0:-1]

    else:
      #label = [{"role": "assisitant", "content" : wrapped_dialogues[-2]['content']}]
      label = str(wrapped_dialogues[-2]['content'])
      original_labels.append(wrapped_dialogues[-2]['content'])
      wrapped_dialogues = wrapped_dialogues[0:-2]
    #wrapped_dialogues = tokenizer.apply_chat_template(wrapped_dialogues,padding=False, truncation=False,  tokenize=tokenize, add_generation_prompt=add_generation_prompt)
    #label = tokenizer.apply_chat_template(label,padding=False, truncation=False,  tokenize=tokenize, add_generation_prompt=False)
    processed_data.append(wrapped_dialogues)
    label_data.append(label)
    output  = pd.DataFrame({"inputs": processed_data,"labels" : label_data})
    #output = pd.DataFrame({"inputs" : processed_data})
    #output  = pd.DataFrame({"inputs": processed_data,"labels" : label_data, "ori_labels" : original_labels})
  return output

In [ ]:
def preprocess_function_1st_reply(Data, tokenize, add_generation_prompt):
  processed_data = []
  label_data  = []
  original_labels = []
  for dialogs in Data:
    flag = 0
    wrapped_dialogues = []
    labels = []
    for dialog in dialogs['dialog']:
      if flag == 0:
        wrapped_dialog = {"role" : "user", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 1
      else:
        wrapped_dialog = {"role" : "assistant", "content" : dialog}
        wrapped_dialogues.append(wrapped_dialog)
        flag = 0

    processed_data.append(wrapped_dialogues[0])
    label_data.append(wrapped_dialogues[1]['content'])
    output  = pd.DataFrame({"inputs": processed_data,"labels" : label_data})
    #output = pd.DataFrame({"inputs" : processed_data})
    #output  = pd.DataFrame({"inputs": processed_data,"labels" : label_data, "ori_labels" : original_labels})
  return output

In [ ]:
test_ds = preprocess_function(dataset['test'], tokenize=False, add_generation_prompt=True)

In [ ]:
test_ds

,inputs,labels
0,"[{'role': 'user', 'content': 'Hey man , you wa...",I want you to put your hands behind your head...
1,"[{'role': 'user', 'content': 'The taxi drivers...",It is really a hot potato .
2,"[{'role': 'user', 'content': 'We've managed to...",What other sources of energy do you use ?
3,"[{'role': 'user', 'content': 'Believe it or no...",Let's go !
4,"[{'role': 'user', 'content': 'What are your pe...","Yes , I like travelling . I am young , and un..."
...,...,...
995,"[{'role': 'user', 'content': 'Frank ’ s gettin...",Have they set a date for the wedding ?
996,"[{'role': 'user', 'content': 'OK . Come back i...","Mam , another minute , could I ?"
997,"[{'role': 'user', 'content': 'Do you have any ...","Yes , stones ."
998,"[{'role': 'user', 'content': 'Jenny , what's w...",Mary told me that she had seen you with John ...


In [ ]:
predictions = []
for i, x in test_ds.iterrows():
  #print(x['inputs'])
  input_ids = tokenizer.apply_chat_template(x['inputs'], truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")

  peft_model_result = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=20, eos_token_id = 50256))

  sliced_peft_model_text_output = tokenizer.decode(peft_model_result[0][input_ids.shape[1]:], skip_special_tokens=True)
  #index = sliced_peft_model_text_output.find("\nuser")

  predictions.append(sliced_peft_model_text_output)


  if len(predictions) == 50:
    break




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
original_model_results_last = rouge.compute(
    predictions=predictions,
    references=test_ds['labels'][0:len(predictions)],
    use_aggregator=True,
    use_stemmer=True,
)
original_model_results_last

{'rouge1': 0.21255654577214209,
 'rouge2': 0.09827546432062562,
 'rougeL': 0.20128463669249905,
 'rougeLsum': 0.20111380883718843}

In [ ]:
df=pd.DataFrame({"inputs": predictions,"labels" : test_ds['labels'][0:len(predictions)]})
df

,inputs,labels
0,"I got some weed, some hash, some acid",I want you to put your hands behind your head...
1,I think they are right. \n,It is really a hot potato .
2,That sounds like a good idea. \n,What other sources of energy do you use ?
3,OK. \nuser\n I'll,Let's go !
4,"Yes, I ’ m available for travel.","Yes , I like travelling . I am young , and un..."
5,I think we will be able to get there in,"we can ’ t . if we went that fast , we would ..."
6,"No, I'm sorry. \nuser",You can take it in a couple weeks .
7,Thank you. \nuser\n,Thank you very much .
8,You're welcome. \nuser\n I,But you must cook that dinner next time .
9,I think I'd be fine. \n,I can't really deal with any distractions rig...
